<a href="https://colab.research.google.com/github/TesisDeepcodeUcuenca/Tesis-Deepcode-Ucuenca/blob/main/C%C3%B3digos%20convencionales/C%C3%B3digos_de_repetici%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Códigos de repetición

import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import csv


def canal_bsc(mensaje, ruido):
    msg_canal_bits = np.bitwise_xor(mensaje, ruido)
    return msg_canal_bits

def calculate_ber(original_bits, decoded_bits):
    error_count = sum([x != y for x, y in zip(original_bits, decoded_bits)])
    ber = error_count / len(original_bits)
    return ber
def guardar_matriz_en_csv(matriz, nombre_archivo):
    # Convertir la matriz a un DataFrame de pandas
    df = pd.DataFrame(matriz.T)  # Transponer para que cada columna corresponda a una probabilidad

    # Asignar nombres a las columnas como Probabilidad 1, Probabilidad 2, etc.
    df.columns = [f'Probabilidad {i + 1}' for i in range(matriz.shape[0])]

    # Guardar el DataFrame en un archivo CSV
    df.to_csv(nombre_archivo, index=False)
def repetition_encoder(message, repeat_factor):
    return np.repeat(message, repeat_factor)
def repetition_decoder(received_bits, repeat_factor):
    decoded_bits = np.reshape(received_bits, (len(received_bits) // repeat_factor, repeat_factor))
    decoded_bits = np.sum(decoded_bits, axis=1) > repeat_factor // 2
    return decoded_bits.astype(int)


def tanda_train(batch_size, block_len, code_rate, qs, _, progreso_tanda):
    ber_q = np.zeros((len(qs), batch_size))
    for j, q in enumerate(qs):
        data = np.random.randint(0, 2, (batch_size, block_len, 1))
        noise = np.random.binomial(1, q, size=(batch_size, (block_len*code_rate) , 1)).astype(int)
        for i in range(0, batch_size):
            progreso_tanda.update(1)
            raw_data = (data[i:i+1, :, :]).squeeze().squeeze()
            batch_noise = (noise[i:i+1, :, :]).squeeze().squeeze()
            encoded_vector = repetition_encoder(raw_data, 3)
            channel_vector = canal_bsc(encoded_vector, batch_noise)
            decoded_vector = repetition_decoder(channel_vector, 3)
            ber = calculate_ber(raw_data, decoded_vector)
            ber_q[j][i] = ber
    return ber_q

def guardar_csv(data, filename):
    filepath = filename  # Ajusta la ruta según dónde quieras guardar el archivo
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Q values', 'Average BER'])
        for row in data:
            writer.writerow(row)



if __name__ == "__main__":
    names = ["repeticion_1_3.csv"]

    for name in names:
        block_len = 50
        batch_size = 100
        code_rate = 3
        qs = np.linspace(0.0001, 0.15, 20)
        num_epoch_test = 100

        arrays_q_x3 = []
        print(f"Total:", num_epoch_test, "tandas")
        progreso_tanda = tqdm(total=num_epoch_test*20*batch_size, desc=f"Progreso GENERAL")
        for _ in range(num_epoch_test):
            ber_q_x3 = tanda_train(batch_size, block_len, code_rate, qs, _, progreso_tanda)
            arrays_q_x3.append(ber_q_x3)
            #progreso_tanda.update(1)

        qs_reshaped = qs.reshape(20, 1)

        x3_stack = np.hstack(arrays_q_x3)
        x3_data = np.hstack((qs_reshaped, x3_stack))
        guardar_matriz_en_csv(x3_data, "repeticion_1_3.csv")
        print(x3_data.shape)